# Task 8.4 Qiskit Runtime Rest API

## Objective 1 : Interact with Qiskit IBM Runtime REST API

The Qiskit Runtime REST API provides programmatic access to IBM Quantum systems, allowing you to:

* **Run circuits** on quantum processors (QPUs)
* **Manage jobs and sessions** with the abiltiy to submit, delete , cancel jobs and retrieve results
* **Get information** about quantum processors, service instances, workloads, accounts and usage details


### Authentication

To use the Qiskit Runtime REST API, you need:
1. **IBM Cloud API Key**: From your IBM Cloud account
2. **IBM Quantum Instance CRN**: From your Quantum service instance

The authentication flow:
1. Exchange API key for IAM token
2. Use IAM token + Instance CRN for API requests

In [ ]:
import requests
import json
 
url = 'https://iam.cloud.ibm.com/identity/token'
api_key="<YOUR_API_KEY>"


data = f'grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={api_key}'
response = requests.post(url, data=data)

# Bearer token to authorize requests to the REST API
if response.status_code == 200:
        print(f"Access token obtained successfully.")
        bearer_token = response.json()['access_token']
else:
    print(f"Error obtaining bearer token:{response.status_code} ,{response.text}")
    bearer_token = None


### Get available Backends

The example below shows how you can retrieve available backends

In [ ]:
import requests
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/backends"

service_CRN = "<crn:YOUR_INSTANCE_CRN>"
 
headersList = {
  "Accept": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version":"2025-05-01"
}

payload = ""
 
response = requests.request("GET", reqUrl, data=payload,  headers=headersList)
 
if response.status_code == 200:
        print(f"Available Backends are:")
        print(response.json())
        # Selecting the first backend from the list
        backend = response.json()['devices'][0]['name']  

else:
    print(f"Error obtaining backends:{response.status_code} ,{response.text}")
    backend = None



### Submit a Job

The example below shows how to submit a job to run a quantum circuit using the sampler primitive


<span style="color:red"> **Running the cell below will use up to 5 seconds from your IBM account credit**
</span>

In [ ]:
import requests
import json
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/jobs"

# Service CRN of your IBM Quantum instance
# bearer token obtained from IAM 
headersList = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version":"2025-05-01"
}

# Sample QASM circuit to be submitted 
circuit_qasm = '''OPENQASM 3.0; include "stdgates.inc"; bit[1] c; x $0; c[0] = measure $0;'''
 
payload = json.dumps({
  "program_id": "sampler",
  # Target backend
  "backend": f"{backend}",
  "params": {
    "pubs": [[
      circuit_qasm
    ]],
    "version": 2
  }
})
 
response = requests.request("POST", reqUrl, data=payload,  headers=headersList)

if response.status_code == 200:
        print(f"Submitted Job:")
        print(response.json())
        job_id = response.json()['id']
else:
    print(f"Error submitting job:{response.status_code} ,{response.text}")
    job_id = None



### Retrieve Results

The example below shows how you can monitor the job status after submitting it, and retrieve the results when it is completed.


In [ ]:
import requests
import json
 
#reqUrl = f"https://quantum.cloud.ibm.com/api/v1/jobs/{job_id}"
reqUrl = f"https://quantum.cloud.ibm.com/api/v1/jobs/d5dngo1smlfc739ohtmg"
results_reqUrl = f"https://quantum.cloud.ibm.com/api/v1/jobs/d5edd5qgim5s73aecqr0/results"
 
headersList = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version":"2025-05-01"
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

if response.status_code == 200:
        print(f"Job Status: {response.json()['status']}")
        # Check if job is completed to retrieve results
        if response.json()['status'] == 'Completed':
            print("Retrieving Results:")
            results_response = requests.request("GET", results_reqUrl, data=payload,  headers=headersList)
            if results_response.status_code == 200:
                print(f"Job Results:")
                print(results_response.json())
            else:
                print(f"Error retrieving job results:{results_response.status_code} ,{results_response.text}")
        elif response.json()['status'] == 'Failed':
            print("Job has failed. No results to retrieve.")
        else:
            print(f"Job is {response.json()['status']}")
else:
    print(f"Error retrieving job status:{response.status_code} ,{response.text}")



### Create a Session

The example below shows how you can create a session and then get the session id to use it later.


<span style="color:red"> **Note:Open plan users can not run job in session execution mode, The following code blocks in the rest of the notebook will fail if you have an open plan** </span>

In [ ]:
import requests
import json
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/sessions"
 
headersList = {
  "Accept": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "Content-Type": "application/json"
}
 
payload = json.dumps({
  "mode": "dedicated",
  "max_ttl": 28800
})
 
response = requests.request("POST", reqUrl, data=payload,  headers=headersList)

if response.status_code == 200:
  sessionId = response.json()['id']
  print(response.json())
  print(sessionId)
else:
    print(f"Error creating session:{response.status_code} ,{response.text}")


### Submit a Job in a session

You can then use the session id to run a job in a session like the example below

In [ ]:
import requests
import json
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/jobs"
 
headersList = {
  "Accept": "application/json",
  "Authorization": f"Bearer {bearer_token}",
  "Service-CRN": f"{service_CRN}",
  "IBM-API-Version": "2025-05-01",
  "Content-Type": "application/json"
}
 
payload = json.dumps({
  "program_id": "sampler",
  "backend": f"{backend}",
  "session_id": f"{sessionId}",
  "params": {
    "pubs": [[
      "OPENQASM 3.0; include \"stdgates.inc\"; bit[1] c; x $0; c[0] = measure $0;"
    ]],
    "options": {},
    "version": 2
  }
})
 
response = requests.request("POST", reqUrl, data=payload,  headers=headersList)
if response.status_code == 200:
  print(f"Submitted Job:")
  print(response.json())
else:
  print(f"Error submitting job:{response.status_code} ,{response.text}")

---

## Practice Questions

**1) Why might the code below fail?**

```
import requests
import json
 
reqUrl = "https://quantum.cloud.ibm.com/api/v1/jobs"

headersList = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": f"Bearer {My_API_token}",
  "Service-CRN": f"{My_Service_CRN}",
  "IBM-API-Version":"2025-05-01"
}

circuit_qasm = '''OPENQASM 3.0; include "stdgates.inc"; bit[1] c; x $0; c[0] = measure $0;'''
 
payload = json.dumps({
  "program_id": "sampler",
  "backend": f"{backend}",
  "params": {
    "pubs": [[
      circuit_qasm
    ]],
    "version": 2
  }
})
 
response = requests.request("POST", reqUrl, data=payload,  headers=headersList)
print(response.json())
job_id = response.json()['id']
```


A) request URL should have the job id

B) The token must be passed as a query parameter

C) The authorization header should have the bearer token not the API token

D) program_id is not required


***Answer:***
<Details>
C) The authorization header should have the Bearer token that is returned after calling the authentication API with your cloud API token
<br/>
</Details>

---

**2) what information will be returned from the request below?**

```
curl -X GET https://quantum.cloud.ibm.com/api/v1/jobs/12345 \
  -H "Authorization: Bearer BEARER_TOKEN",
  -H "Service-CRN": f"{service_CRN}"
```

A) Job results only

B) Backend calibration data

C) The transpiled circuit and its execution parameters

D) Job metadata and execution status

***Answer:***
<Details>
D) The response will have the job details like status , backend , creation and run times, ...
<br/>
</Details>

---